### ASGI.py & WSGI.py

 - wsgi.py --> Used for synchronous application, traditional synchronous interface between Django and web servers like gunicorn, uWSGI, Apache etc. Works well with traditional http requests.
 - Standard way to connect python web apps to web servers (Gunicorn, uWSGI)
 - When we deploy django project on production, it acts like entrypoint, web server like Gunicorn uses this file to connect itself to Django.
 - It has application named callable object which handles the request and generates the response.
 - WSGI is synchronous so it doesn't support real time features like Websockets.

 `gunicorn myproject.wsgi:application`

In [ ]:
import os
from django.core.wsgi import get_wsgi_application

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'myproject.settings')

application = get_wsgi_application()

# get_wsgi_application() creates a WSGI application object
# Web servers call application to process requests.

# Using wsgi in production:
# gunicorn myproject.wsgi.application

 - asgi.py --> Used for asynchronous features, modern replacement for WSGI. It allows django to handle asynchronous requests. Used for chat apps, notifications etc.
 - ASGI Servers like Uvicorn, Daphne uses this file to work with Django so they can handle both synchronous & aynchronous requests.

In [ ]:
import os
from django.core.asgi import get_asgi_application

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'myproject.settings')

application = get_asgi_application()

# get_asgi_application() cerates a ASGI application object
# Works with Daphne/Uvicorn instead of WSGI servers.
# uvicorn myproject.asgi:application --host 0.0.0.0 --port 8000

#### Role of Gunicorn & Nginx:
- Gunicorn is a WSGI server which runs Django application. It executes Python code and returns response after processing HTTP requests.
- It doesn't directly handle client requests. To directly handle client requests, we use Nginx.
- Nginx is a web server acting like reverse proxy taking client request, forwards it to Gunicorn.
- Gunicorn executes the application logic and Nginx manages the traffic.
- Gunicorn = App runner, Nginx = Proxy + Static Files.

#### Why apps.py is used in Django
 - Registers the application in Django project
 - Custom app configurations (app name, define signals)

In [ ]:
from django.apps import AppConfig

class MyappConfig(AppConfig):
    name = 'myapp'

    def ready(self):
        import myapp.signals  # Import signals when the app is ready


#### Sessions in Django:
- Way to store user specific data between requests.
- Django uses a session framework that works as follows:
    - User makes a request --> Django assigns a unique session ID
    - Session data is stored --> this data is stored on Server, and session ID is stored in user's browser as cookie (sessionId)
    - On subsequent requests --> Django retrieves session data using SessionID
- Django sessions are enabled by default in settings.py
    - INSTALLEDA_APPS = ['django.contrib.sessions']
    - MIDDLEWARE = ['django.middleware.session.SessionMiddleWare']

#### Static Files in Django:
- Static files in Django include CSS, JavaScripts, Images, Fonts that doesn't change dynamically. 
- We configure nginx on production, so it can also serve static files directly from STATIC_ROOT because Django doesn't serve static_files in production (DEBUG=FALSE)

In [ ]:
import os

# URL for serving static files
STATIC_URL = '/static/'

# App Directory from where static files are collected
STATICFILES_DIRS = [
    os.path.join(BASE_DIR, 'static'),  # Specifies additional static file locations
]

# Location where collected static files are stored (used in production)
STATIC_ROOT = os.path.join(BASE_DIR, 'staticfiles')

In production, Django does not server static files, Use a web server like Nginx or AWS S3.

`location /static/ {
    root /path/to/your/project/staticfiles;
}`

#### Setting Up Database in Production
- We use PostgreSQL/MySQL as they're scalable & reliable.

#### Reverse Proxy:
- It's like a middleman sitting between client & server (Django App). Client sends request to reverse proxy and it forwards the request to Server, and forwards the response from server to client.
- `Normal Proxy`: (VPN) (Outbound --> From client to outside world) It fulfils the needs of client side and hides client identity from the server, when client wants to talk to server. Normal Proxy takes my request to server. Server doesn't know who the client is, it just sees the proxy.
  - Privacy: Server doesn't know the IP address of client, Server just knows Proxy's IP.
  - Caching: If Proxy already has response, it returned the response without increasing load on server.
- `Reverse Proxy`:(Inbound traffic --> From outside to server) Works from server's side, forwards request from client to server. Accepts the request from client and decides which server it shall forward the request to. Client doesn't know who the actual server is, it's only dealing with reverse proxy.
  - Server Protection: Gunicorn gets hide and client can only see the reverse proxy.
  - Load Balancing: If there are 5 Gunicorn servers, reverse proxy distributes the load.
  - Static Files: Directly serves the static file and doesn't increase the load on server.
  - Single Entry Point: Client only knows one address, though many servers could be running on backend.

In our example, Nginx is reverse proxy, Gunicorn is server and Django runs behind gunicorn.
- Client sends a request to http:<my-ip>.
- Nginx(reverse proxy) listens to this request, as it's running on port-80
- Nginx lookups in it's conf file and serves static file if the request is for static file (eg: /static/style.css), location /static/
- If request is for application (eg: /), then it forwards to Gunicorn (proxy_pass http://web:8000)
- Gunicorn takes the request to Django app, Django generates the response and is returned to Nginx by Gunicorn. Nginx finally returned response to Nginx.

Why to use Reverse Proxy:
- Problems without Reverse Proxy:
    - Gunicorn doesn't serve static files efficiently.
    - Gunicorn is not efficient in handling multiple requests.
    - Gunicorn port will be publicly accessible, security is compromised.
- Nginx as Reverse Proxy:
    - Serves the static files directly
    - Hides Gunicorn and only Nginx is publicly accessible
    - It's fast and scalable.
    - Can helps in balancing the load to different servers if there are many running servers available.